__CORE REQUEST 2__

In [7]:
import pandas as pd
from datetime import datetime
import time
import matplotlib.pyplot as plt
import json
import folium
from branca.colormap import linear


In [8]:
geo_json_data = json.load(open('taxi_zones.json'))

In [9]:
# retieving data to be cleaned
raw_data  = pd.read_csv("yellow_tripdata_2018-01.csv")

# cleaning data:
# initial time equal final time
# distance <= 0
# total amount < 0 
# year not 2018
# month not January (change 1 with the appropiate month)
# trip duration > 200 minutes
# trip distance > 200 miles

raw_data['delta'] = (pd.to_datetime(raw_data['tpep_dropoff_datetime']) -  pd.to_datetime(raw_data['tpep_pickup_datetime'])).astype('timedelta64[m]')
data = raw_data.drop(raw_data[(raw_data.delta > 200) | (raw_data.trip_distance <= 0) | (raw_data.trip_distance > 200) | (raw_data.tpep_dropoff_datetime == raw_data.tpep_pickup_datetime) | (raw_data.total_amount < 0) | (pd.DatetimeIndex(raw_data['tpep_pickup_datetime']).month != 1) | (pd.DatetimeIndex(raw_data['tpep_pickup_datetime']).year != 2018)].index) 

# free memory deleting the raw_data
del raw_data

In [51]:
data['PULocationID_2'] = data['PULocationID']

data_pu = data.set_index('PULocationID')['PULocationID_2']

colormap = linear.OrRd_09.scale(
    data_pu.value_counts().tail(1).item(),
    data_pu.value_counts().head(1).item())

pu = pd.DataFrame(data_pu.value_counts())
pu_dict = pu.set_index(pu.index)['PULocationID_2']

def daje_pu(feature):
    try:
        return pu_dict[feature['properties']['LocationID']]
    except:
        return 176
    
m = folium.Map([40.7, -74], tiles='cartodbpositron', zoom_start=10.5)

folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        'fillColor': colormap(daje_pu(feature)),
        'color': 'black',
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
).add_to(m)
m.save('pickup.html')

In [50]:
data['DOLocationID_2'] = data['DOLocationID']
data_do = data.set_index('DOLocationID')['DOLocationID_2']

colormap_do = linear.OrRd_09.scale(
    data_do.value_counts().tail(1).item(),
    data_do.value_counts().head(1).item())

do = pd.DataFrame(data_do.value_counts())
do_dict = do.set_index(do.index)['DOLocationID_2']

def daje_do(feature):
    try:
        return do_dict[feature['properties']['LocationID']]
    except:
        return 2
    
n = folium.Map([40.7, -74], tiles='cartodbpositron', zoom_start=10.5)

folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        'fillColor': colormap_do(daje_do(feature)),
        'color': 'black',
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
).add_to(n)
n.save('dropoff.html')
